In [ ]:
import temet
import h5py
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

from scale_factor_helpers import *

In [ ]:
def load_spectra_from_boxes(gp_paths, spectrumN):
    
    wavelengths = []
    fluxes = []
    params = []
    
    for gp_path in gp_paths:
        gp_name = gp_path.split("/")[-2]
        path = gp_path + f"data.files/spectra/spectra_{gp_name}_z2.0_n100d2-fullbox_SDSS-BOSS_HI_combined.hdf5"

        with h5py.File(path, "r") as f:     
            # Zugriff auf einzelne Datasets
            wavelengths.append(f["wave"][:])  # Das [:] liest das gesamte Dataset in ein NumPy-Array ein
            fluxes.append(f["flux"][:][spectrumN])

        outdir_path = gp_path + "output/"
        Omega0, OmegaBaryon, OmegaLambda, HubbleParam = get_cosmo_parameters(outdir_path)

        params.append([Omega0, OmegaBaryon, OmegaLambda, HubbleParam])

    return wavelengths, fluxes, params


def get_cosmo_parameters(basepath):
    path = basepath+"txt-files/parameters-usedvalues"
    Omega0 = None
    OmegaLambda = None
    HubbleParam = None
    OmegaBaryon = None
    
    with open(path, "r") as f:
        for line in f:
            if "Omega0" in line:
                Omega0 = float(line.split()[-1])
            if "OmegaBaryon" in line:
                OmegaBaryon = float(line.split()[-1])
            if "OmegaLambda" in line:
                OmegaLambda = float(line.split()[-1])
            if "HubbleParam" in line:
                HubbleParam = float(line.split()[-1])
    
    return Omega0, OmegaBaryon, OmegaLambda, HubbleParam


def redshift_wavelength_forward(z, wavelength):
    return (z+1)*wavelength


def redshift_wavelength_backward(z, wavelength):
    return wavelength/(z+1)

In [ ]:
gps_list = [i for i in range(50)]

base_path = "/vera/u/jerbo/my_ptmp/L25n256_suite"
gp_paths = [base_path+f"/gridpoint{i}/" for i in gps_list]

spectrum_num = 9000
wavelengths, fluxes, params = load_spectra_from_boxes(gp_paths, spectrum_num)

Ly_alpha_0 = 1215.67  # in Angström
z_box = 2
boxsize = 25  # in cMpc

Ly_alphas_z = []

for i in range(len(wavelengths)):
    wavelength_current = wavelengths[i]
    (Omega_m, Omega_b, Omega_Lambda, h) = params[i]
    dz = delta_z(boxsize, z_box, Omega_m, Omega_Lambda, h)
    corr_z = corrected_z(boxsize, z_box, Omega_m, Omega_Lambda, h)
    # print(dz)
    wavelengths[i] = redshift_wavelength_backward(corr_z, wavelength_current)


# print(Ly_alphas_z)


In [ ]:
fig, axe = plt.subplots()

cosmo_par_index = 1  # 0: Omega0, 1: OmegaBaryon, 2: OmegaLambda, 3: HubbleParam
cosmo_par = [i[cosmo_par_index] for i in params]

norm = colors.Normalize(vmin=min(cosmo_par), vmax=max(cosmo_par))
# colormap = cm.viridis  # Choose any colormap: viridis, plasma, inferno, etc.
scalar_map = cm.ScalarMappable(norm=norm, cmap="jet")

for i in range(35, len(wavelengths)-10):
    color = scalar_map.to_rgba(cosmo_par[i])
    plt.plot(wavelengths[i], fluxes[i], alpha=0.7, c=color)

plt.plot([Ly_alpha_0, Ly_alpha_0],[-0.1, 1.1], c="black", linestyle="--")
plt.annotate(r"$Ly \alpha$", [Ly_alpha_0-1.5, 1.05], rotation=90)

cbar = plt.colorbar(scalar_map, ax=axe)
cbar.set_label(r'$\Omega_\text{b}$')
plt.xlabel(r"Wavelength in $\AA$")
plt.ylabel(r"Relative flux ")
plt.xlim([1200, 1240])
plt.show()


